In [1]:
!pip install -q transformers bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen3-8B"

# load the tokenizer and the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map=None,
    load_in_4bit = True
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
prompt="""
You are playing Wordle, a word-guessing game.

Game rules:
- You have 6 tries to guess a secret 5-letter word.
- Each guess must be a valid 5-letter English word.
- After each guess, you receive feedback per letter:
  “✓” means the letter is correct and in the right position,
  “-” means the letter is in the word but in a different position,
  “x” means the letter is not in the word.

Here is the current game state:
Word list: ["CRANE", "SLATE", "APPLE", "GRACE", "PLANT", "MIGHT", "BRINK", "PROUD", "SMILE", "TRAIL"]
Past guesses & feedback:
1. CRANE → C(x) R(-) A(x) N(x) E(x)
2. SLATE → S(x) L(x) A(x) T(x) E(x)

Your next move: Think step by step and then guess the next word.
<think>
"""

In [6]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generation configuration
gen_output = model.generate(
    **inputs,
    max_new_tokens=2000,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

# Decode model output
decoded = tokenizer.decode(gen_output[0], skip_special_tokens=True)
print("\n🧠 Model Output:\n")
print(decoded)


🧠 Model Output:


You are playing Wordle, a word-guessing game.

Game rules:
- You have 6 tries to guess a secret 5-letter word.
- Each guess must be a valid 5-letter English word.
- After each guess, you receive feedback per letter:
  “✓” means the letter is correct and in the right position,
  “-” means the letter is in the word but in a different position,
  “x” means the letter is not in the word.

Here is the current game state:
Word list: ["CRANE", "SLATE", "APPLE", "GRACE", "PLANT", "MIGHT", "BRINK", "PROUD", "SMILE", "TRAIL"]
Past guesses & feedback:
1. CRANE → C(x) R(-) A(x) N(x) E(x)
2. SLATE → S(x) L(x) A(x) T(x) E(x)

Your next move: Think step by step and then guess the next word.
<think>
Okay, let's try to figure out the next word to guess. So, the secret word is a 5-letter word, and I have to use the clues from the previous guesses to narrow it down.

First, let me recap the feedback from the past guesses. 

First guess was CRANE. The feedback was C(x), R(-), A(x), N(x)

In [9]:
prompt="""
You are playing Wordle, a word-guessing game.

Game rules:
- You have 6 tries to guess a secret 5-letter word.
- Each guess must be a valid 5-letter English word.
- After each guess, you receive feedback per letter:
  “✓” means the letter is correct and in the right position,
  “-” means the letter is in the word but in a different position,
  “x” means the letter is not in the word.

Here is the current game state:
Word list: ["CRANE", "SLATE", "PROUD", "GRACE", "PLANT", "MIGHT", "BRINK", "ROUND", "SMILE", "TRAIL"]
Past guesses & feedback:
1. CRANE → C(x) R(-) A(x) N(✓) E(x)
2. SLATE → S(x) L(x) A(x) T(x) E(x)
3. PROUD → S(x) R(-) O(-) U(-) D(✓)

Your next move: Think step by step and then guess the next word.
"""

In [12]:
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2000
)

In [11]:
decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("\n🧠 Model Output:\n")
print(decoded)


🧠 Model Output:

user

You are playing Wordle, a word-guessing game.

Game rules:
- You have 6 tries to guess a secret 5-letter word.
- Each guess must be a valid 5-letter English word.
- After each guess, you receive feedback per letter:
  “✓” means the letter is correct and in the right position,
  “-” means the letter is in the word but in a different position,
  “x” means the letter is not in the word.

Here is the current game state:
Word list: ["CRANE", "SLATE", "PROUD", "GRACE", "PLANT", "MIGHT", "BRINK", "ROUND", "SMILE", "TRAIL"]
Past guesses & feedback:
1. CRANE → C(x) R(-) A(x) N(✓) E(x)
2. SLATE → S(x) L(x) A(x) T(x) E(x)
3. PROUD → S(x) R(-) O(-) U(-) D(✓)

Your next move: Think step by step and then guess the next word.

assistant
<think>
Okay, let's try to figure out the next word to guess in Wordle. So, the secret word is a 5-letter English word, and I have to use the clues from the previous guesses and their feedback to narrow it down.

First, let me list out the curr